In [ ]:
import logging

import shapely
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr

from searvey import ioc

logging.basicConfig(
    level=20,
    style="{",
    format="{asctime:s}; {levelname:8s}; {threadName:23s}; {name:<25s} {lineno:5d}; {message:s}",
)

logging.getLogger("urllib3").setLevel(30)
logging.getLogger("parso").setLevel(30)

logger = logging.getLogger(__name__)

## Retrieve Station Metadata

In [ ]:
ioc_stations = ioc.get_ioc_stations()
ioc_stations

In [ ]:
figure, axis = plt.subplots(1, 1)
figure.set_size_inches(12, 12 / 1.61803398875)

countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
_ = countries.plot(color='lightgrey', ax=axis, zorder=-1)
_ = ioc_stations.plot(ax=axis)
_ = axis.set_title(f'all IOC stations')

In [ ]:
ioc_stations.columns

## Retrieve station metadata from arbitrary polygon

In [ ]:
east_coast = shapely.geometry.box(-85, 25, -65, 45)
east_coast

east_stations = ioc.get_ioc_stations(region=east_coast)
east_stations

In [ ]:
east_stations[~east_stations.contacts.str.contains("NOAA")]

## Retrieve IOC station data

In [ ]:
east_data = ioc.get_ioc_data(
    ioc_metadata=east_stations,
    endtime="2020-05-30",
    period=3,
)
east_data

In [ ]:
def drop_all_nan_vars(ds: xr.Dataset) -> xr.Dataset:
    for var in ds.data_vars:
        if ds[var].notnull().sum() == 0:
            ds = ds.drop_vars(var)
    return ds

ds = drop_all_nan_vars(east_data.sel(ioc_code="setp1"))
ds

In [ ]:
fix, axes = plt.subplots(1, 1)

_ = ds.prs.plot(ax=axes)
_ = ds.rad.plot(ax=axes)
_ = ds.ra2.plot(ax=axes)

In [ ]:
ds.where(ds.country == "Bahamas")